In [1]:
import os
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import (DataLoader, TensorDataset)
from sklearn.metrics import accuracy_score

In [2]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'
device = 'mps' if torch.backends.mps.is_available() else device

In [3]:
class SpacingRNN(nn.Module):

    def __init__(self, config):
        super(SpacingRNN, self).__init__()
        
        # (batch_size, max_length) -> (batch_size, max_length, embedding_size)
        self.embedding = 
        # (batch_size, max_length, embedding_size) -> (batch_size, max_length, hidden_size*2)
        self.bi_lstm = 
        # (batch_size, max_length, hidden_size*2) -> (batch_size, max_length, number_of_labels)
        self.linear = 
        
    def forward(self, inputs):
        eumjeol_inputs = self.embedding(inputs)
        hidden_outputs, _ = self.bi_lstm(eumjeol_inputs)
        hypothesis = self.linear(hidden_outputs)
        
        return hypothesis

In [4]:
def read_datas(file_path):
    with open(file_path, "r", encoding="utf8") as inFile:
        lines = inFile.readlines()
    datas = []
    # sample data: 그 사 람 을 아 직 도 사 랑 하 나 요 ?\t B B I I B I I B I I I I I 
    for line in lines:
        # ['그 사 람 을 아 직 도 사 랑 하 나 요 ?', 'B B I I B I I B I I I I I']
        # ['그', '사', '람', '을', '아', '직', '도', '사', '랑', '하', '나', '요', '?'],
        # ['B', 'B', 'I', 'I', 'B', 'I', 'I', 'B', 'I', 'I', 'I', 'I', 'I']
        datas.append((eumjeol_list, label_list))
    return datas

In [5]:
def read_vocab_data(eumjeol_vocab_data_path):
    label2idx, idx2label = {"<PAD>":0, "B":1, "I":2}, {0:"<PAD>", 1:"B", 2:"I"}
    eumjeol2idx, idx2eumjeol = {}, {}

    with open(eumjeol_vocab_data_path, "r", encoding="utf8") as inFile:
        lines = inFile.readlines()

    for line in lines:
        eumjeol = line.strip()
        eumjeol2idx[eumjeol] = len(eumjeol2idx)
        idx2eumjeol[eumjeol2idx[eumjeol]] = eumjeol

    return eumjeol2idx, idx2eumjeol, label2idx, idx2label

In [7]:
def load_dataset(config, is_train=True):
    datas = read_datas(config["input_data"])
    eumjeol2idx, idx2eumjeol, label2idx, idx2label = read_vocab_data(config["eumjeol_vocab"])
    
    eumjeol_features, eumjeol_feature_lengths, label_features = [], [], []
    
    for eumjeol_sequence, label_sequence in datas:
        eumjeol_feature = [eumjeol2idx[eumjeol] for eumjeol in eumjeol_sequence]
        label_feature = [label2idx[label] for label in label_sequence]
        
        eumjeol_feature_length = len(eumjeol_feature)
        
        eumjeol_feature +=  
        label_feature += 
        
        eumjeol_features.append(eumjeol_feature)
        eumjeol_feature_lengths.append(eumjeol_feature_length)
        label_features.append(label_feature)
        
    if is_train:
        return torch.tensor(eumjeol_features, dtype=torch.long), \
               torch.tensor(eumjeol_feature_lengths, dtype=torch.long), \
               torch.tensor(label_features, dtype=torch.long)
    else:
        return torch.tensor(eumjeol_features, dtype=torch.long), \
           torch.tensor(eumjeol_feature_lengths, dtype=torch.long), \
           torch.tensor(label_features, dtype=torch.long), idx2eumjeol, idx2label

In [8]:
def train(config):
    model = SpacingRNN(config).to(device)
    eumjeol_features, eumjeol_feature_lengths, label_features = load_dataset(config)
    train_features = TensorDataset(eumjeol_features, eumjeol_feature_lengths, label_features)
    train_dataloader = DataLoader(train_features, shuffle=True, batch_size=config["batch_size"])
    loss_func = nn.CrossEntropyLoss()
    optimizer = optim.Adam(model.parameters(), lr=0.001)

    for epoch in range(config["epoch"]):
        model.train()
        costs = []
        for step, batch in enumerate(train_dataloader):
            optimizer.zero_grad()
            batch = tuple(t.to(device) for t in batch)
            inputs, input_lengths, labels = batch[0], batch[1], batch[2]
            
            hypothesis = 
            cost = 
            cost.backward()
            optimizer.step()
            # batch 단위 cost 값 저장
            costs.append(cost.data.item())
            
            # 10 step 마다 loss 값 출력
            if (step+1) % 10 == 0:
                print("Epoch : {0:d}, Step : {1:d}, Cost : {2:.6f}".format(epoch + 1, step+1, cost.data.item()))

        torch.save(model.state_dict(), os.path.join(output_dir, "epoch_{0:d}.pt".format(epoch + 1)))

        # epoch 별로 평균 loss 값 출력
        print("Epoch : {0:d}, Average Cost : {1:.6f}".format(epoch + 1, np.mean(costs)))

In [9]:
def make_sentence(inputs, predicts, labels, idx2eumjeol, idx2label):

    predict_sentence, correct_sentence = "", ""

    for index in range(len(inputs)):
        eumjeol = idx2eumjeol[inputs[index]]
        correct_label = idx2label[labels[index]]
        predict_label = idx2label[predicts[index]]

        if (index == 0):
            predict_sentence += eumjeol
            correct_sentence += eumjeol
            continue

        if (predict_label == "B"):
            predict_sentence += " "
        predict_sentence += eumjeol

        if (correct_label == "B"):
            correct_sentence += " "
        correct_sentence += eumjeol

    return predict_sentence, correct_sentence

def tensor2list(input_tensor):
    return input_tensor.cpu().detach().numpy().tolist()

def test(config):
    eumjeol_features, eumjeol_feature_lengths, label_features, idx2eumjeol, idx2label = load_dataset(config, False)

    test_features = TensorDataset(eumjeol_features, eumjeol_feature_lengths, label_features)
    test_dataloader = DataLoader(test_features, shuffle=False, batch_size=1)

    model = SpacingRNN(config).to(device)
    model.load_state_dict(torch.load(os.path.join(config["output_dir_path"], config["model_name"])))

    total_hypothesis, total_labels = [], []

    for step, batch in enumerate(test_dataloader):

        model.eval()
        batch = tuple(t.to(device) for t in batch)

        inputs, input_lengths, labels = batch[0], batch[1], batch[2]

        hypothesis = model(inputs)

        hypothesis = torch.argmax(hypothesis, dim=-1)

        input_length = tensor2list(input_lengths[0])
        input = tensor2list(inputs[0])[:input_length]
        label = tensor2list(labels[0])[:input_length]
        hypothesis = tensor2list(hypothesis[0])[:input_length]

        total_hypothesis += hypothesis
        total_labels += label

        if (step < 10):
            predict_sentence, correct_sentence = make_sentence(input, hypothesis, label, idx2eumjeol, idx2label)
            print("정답 : " + correct_sentence)
            print("출력 : " + predict_sentence)
            print()

    print("Accuracy : {}".format(accuracy_score(total_labels, total_hypothesis)))

In [10]:
if(__name__=="__main__"):
    root_dir = "."
    output_dir = os.path.join(root_dir, "output")
    if not os.path.exists(output_dir):
        os.makedirs(output_dir)

    config = {"mode": "train",
              "model_name":"epoch_{0:d}.pt".format(5),
              "input_data":os.path.join(root_dir, "train-2.txt"),
              "output_dir_path":output_dir,
              "eumjeol_vocab": os.path.join(root_dir, "eumjeol_vocab.txt"),
              "label_vocab": os.path.join(root_dir, "label_vocab.txt"),
              "eumjeol_vocab_size": 2458,
              "embedding_size": 100,
              "hidden_size": 100,
              "max_length": 920,
              "number_of_labels": 3,
              "epoch":5,
              "batch_size":64,
              "dropout":0.3
              }

    if(config["mode"] == "train"):
        train(config)
    else:
        test(config)

Epoch : 1, Step : 10, Cost : 0.790247
Epoch : 1, Step : 20, Cost : 0.053088
Epoch : 1, Step : 30, Cost : 0.042955
Epoch : 1, Step : 40, Cost : 0.030374
Epoch : 1, Step : 50, Cost : 0.027951
Epoch : 1, Step : 60, Cost : 0.022417
Epoch : 1, Step : 70, Cost : 0.023687
Epoch : 1, Average Cost : 0.194883
Epoch : 2, Step : 10, Cost : 0.020986
Epoch : 2, Step : 20, Cost : 0.021146
Epoch : 2, Step : 30, Cost : 0.023556
Epoch : 2, Step : 40, Cost : 0.030308
Epoch : 2, Step : 50, Cost : 0.019190
Epoch : 2, Step : 60, Cost : 0.021448
Epoch : 2, Step : 70, Cost : 0.021140
Epoch : 2, Average Cost : 0.020738
Epoch : 3, Step : 10, Cost : 0.020440
Epoch : 3, Step : 20, Cost : 0.019457
Epoch : 3, Step : 30, Cost : 0.018071
Epoch : 3, Step : 40, Cost : 0.017924
Epoch : 3, Step : 50, Cost : 0.016297
Epoch : 3, Step : 60, Cost : 0.014533
Epoch : 3, Step : 70, Cost : 0.017216
Epoch : 3, Average Cost : 0.017471
Epoch : 4, Step : 10, Cost : 0.013262
Epoch : 4, Step : 20, Cost : 0.013349
Epoch : 4, Step : 30,

In [11]:
if(__name__=="__main__"):
    root_dir = "."
    output_dir = os.path.join(root_dir, "output")
    if not os.path.exists(output_dir):
        os.makedirs(output_dir)

    config = {"mode": "test",
              "model_name":"epoch_{0:d}.pt".format(5),
              "input_data":os.path.join(root_dir, "train-2.txt"),
              "output_dir_path":output_dir,
              "eumjeol_vocab": os.path.join(root_dir, "eumjeol_vocab.txt"),
              "label_vocab": os.path.join(root_dir, "label_vocab.txt"),
              "eumjeol_vocab_size": 2458,
              "embedding_size": 100,
              "hidden_size": 100,
              "max_length": 920,
              "number_of_labels": 3,
              "epoch":5,
              "batch_size":64,
              "dropout":0.3
              }

    if(config["mode"] == "train"):
        train(config)
    else:
        test(config)

정답 : 부인이 정성들여 키운 진이 독살되었다는 것은 연박사에게도 매우 가슴아픈 일일 것이다.
출력 : 부인이 정성들여키운진이 독살되었다는 것은 연박사에게도 매우가 슴아픈일일 것이다.

정답 : 이같은 그의 가난은 어려서만이 아니라 자란 뒤에도 늙을 때까지 줄기차게 계속되었다.
출력 : 이 같은 그의 가난은 어려서만이 아니라자란 뒤에도 늙을 때까지줄기차게 계속되었다.

정답 : 어느 물리학자의 머리 속.
출력 : 어느 물리학자의 머리속.

정답 : 말씨며 움직임이 세련되어 있었다.
출력 : 말 씨며 움직임이 세련 되어 있었다.

정답 : "워싱턴과 뉴욕에서 차출한 인원까지 모두 12 명이네."
출력 : "워싱턴과 뉴욕에서 차출한 인원까지 모두 12명이네."

정답 : "그 사람을 아직도 사랑하나요?"
출력 : " 그사람을 아직도 사랑하나요?"

정답 : "나한테 걸린 건 행복한 거예요. 나를 훔치려는 사내들이 많아요. 무슨 말인지 알아요?"
출력 : "나한테 걸린건행복한 거예요. 나를 훔치려는 사내들이 많아요. 무슨 말인지 알아요?"

정답 : 김광민은 집에 돌아와 있었으나 그 얘기를 바로 주남 마을과 녹동 마을에서 일어난 일이었기 때문에 자세히 들을 수 있었다.
출력 : 김 광민은 집에 돌아와 있었으나 그 얘기를 바로 주남마을 과 녹동마을에서 일어난 일이었기 때문에 자세히들을 수 있었다.

정답 : 미리 이야기해 두는 게 좋을 것 같아."
출력 : 미리이야기해 두는게 좋을 것 같아."

정답 : "부인의 입장에서 남편의 자살을 얘기하는 말투로는 너무 냉정하다는 생각 안 드세요?"
출력 : "부인의 입장에서 남편의 자살을 얘기하는 말투로는 너무냉정하다는 생각안드세요?"

Accuracy : 0.8894696490288663
